## IBM Applied Data Science Capstone Course by Coursera¶
## Oslo Bars Final Report
### Opening a new bar in Oslo, Norway

Build a dataframe of neighborhoods in Oslo by web scraping the data from Wikipedia page
Get the geographical coordinates of the neighborhoods
Obtain the venue data for the neighborhoods from Foursquare API
Explore and cluster the neighborhoods
Select the best cluster to open a new bar

### Import necessary libaraies

In [3]:
pip install geocoder

     |████████████████████████████████| 102kB 6.2MB/s ta 0:00:011
     |████████████████████████████████| 829kB 5.8MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/d2/3d/fa76db83bf75c4f8d338c2fd15c8d33fdd7ad23a9b5e57eb6c5de26b430e/click-7.1.2-py2.py3-none-any.whl
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install geopy --user

     |████████████████████████████████| 112kB 8.1MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install BeautifulSoup4 

     |████████████████████████████████| 122kB 7.2MB/s eta 0:00:01     |████████▌                       | 30kB 21.7MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files
import bs4

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup as bsoup# library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


In [2]:
%matplotlib inline 

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs
import geocoder # to get coordinates

### Scrap data from Wikipedia page into a DataFrame

In [3]:
# send the GET request
url='https://en.wikipedia.org/wiki/List_of_boroughs_of_Oslo'
r=requests.get(url)

In [4]:
# Parse the htlm with Soup
page=bsoup(r.text,"html.parser")


In [5]:
rtable=page.table
results=rtable.find_all('tr') 
nrows=len(results)
nrows

16

In [6]:
# iteration loop to harvest all records

records =[]
n=1
while n < nrows :
    Neighborhood=results[n].text.split('\n')[1]
    
    records.append((Neighborhood))
    n=n+1

df=pd.DataFrame(records, columns=['Neighborhood'])
df

,Neighborhood
0,Alna
1,Bjerke
2,Frogner
3,Gamle Oslo
4,Grorud
5,Grünerløkka
6,Nordre Aker
7,Nordstrand
8,Sagene
9,St. Hanshaugen


In [7]:
df.shape

(15, 1)

## Get the geographical coordinates

In [8]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Oslo, Norway'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [9]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in df["Neighborhood"].tolist() ]
coords

[[59.92680000000007, 10.845700000000022],
 [59.942210000000046, 10.811830000000043],
 [59.91886000000005, 10.702390000000037],
 [59.90624000000008, 10.771470000000022],
 [59.96058000000005, 10.884920000000022],
 [59.924330000000054, 10.761190000000056],
 [59.94933000000003, 10.731400000000065],
 [59.86075000000005, 10.80260000000004],
 [59.93868000000003, 10.757630000000063],
 [59.92679000000004, 10.741110000000049],
 [59.96279000000004, 10.920830000000024],
 [59.83342000000005, 10.807780000000037],
 [59.92186000000004, 10.637240000000077],
 [59.94805000000008, 10.642030000000034],
 [59.87992000000003, 10.825840000000028]]

In [11]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']
# check the neighborhoods and the coordinates
print(df.shape)
df


(15, 3)


,Neighborhood,Latitude,Longitude
0,Alna,59.92680,10.84570
1,Bjerke,59.94221,10.81183
2,Frogner,59.91886,10.70239
3,Gamle Oslo,59.90624,10.77147
4,Grorud,59.96058,10.88492
5,Grünerløkka,59.92433,10.76119
6,Nordre Aker,59.94933,10.73140
7,Nordstrand,59.86075,10.80260
8,Sagene,59.93868,10.75763
9,St. Hanshaugen,59.92679,10.74111


In [13]:
### Create a map of Oslo with neighborhoods superimposed on top

In [14]:
# get the coordinates of Oslo, Norway
address = 'Oslo, Norway'

geolocator = Nominatim(user_agent="geopy.geocoders.options.default_user_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Oslo, Norway {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Oslo, Norway 59.9133301, 10.7389701.


In [15]:
# create map of Oslo using latitude and longitude values
map_oslo = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_oslo)  
    
map_oslo

In [17]:
# save the map as HTML file
map_oslo.save('map_oslo.html')

###  Use the Foursquare API to explore the neighborhoods

In [18]:
CLIENT_ID = 'ADXSH1JBMDUJLS4HQS3Z23VSMZJ5QOK0IXMDAWIJR10WJ1WN' # your Foursquare ID
CLIENT_SECRET = '4R3VD1RB5TLYHS1U1FD5M4N25YT1NI5JPFZ2VY53QPIGI2VZ' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ADXSH1JBMDUJLS4HQS3Z23VSMZJ5QOK0IXMDAWIJR10WJ1WN
CLIENT_SECRET:4R3VD1RB5TLYHS1U1FD5M4N25YT1NI5JPFZ2VY53QPIGI2VZ


### Now, let's get the top 100 venues that are within a radius of 2000 meters.

In [19]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [20]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(840, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Alna,59.9268,10.8457,Vinmonopolet (Alnabru),59.925580,10.847860,Wine Shop
1,Alna,59.9268,10.8457,Skeidar,59.925487,10.846883,Furniture / Home Store
2,Alna,59.9268,10.8457,SATS,59.916591,10.848330,Gym / Fitness Center
3,Alna,59.9268,10.8457,XXL Sport & Villmark,59.925614,10.847767,Sporting Goods Shop
4,Alna,59.9268,10.8457,Serrano's,59.925757,10.846810,Spanish Restaurant


### Let's check how many venues were returned for each neighorhood

In [21]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Alna,47,47,47,47,47,47
Bjerke,40,40,40,40,40,40
Frogner,100,100,100,100,100,100
Gamle Oslo,100,100,100,100,100,100
Grorud,18,18,18,18,18,18
Grünerløkka,100,100,100,100,100,100
Nordre Aker,58,58,58,58,58,58
Nordstrand,31,31,31,31,31,31
Sagene,100,100,100,100,100,100


### Let's find out how many unique categories can be curated from all the returned venues

In [22]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 149 uniques categories.


In [23]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Wine Shop', 'Furniture / Home Store', 'Gym / Fitness Center',
       'Sporting Goods Shop', 'Spanish Restaurant', 'Café',
       'Arts & Crafts Store', 'Grocery Store', 'Hotel', 'Racetrack',
       'Paper / Office Supplies Store', 'Bowling Alley',
       'Sushi Restaurant', 'Automotive Shop', 'Electronics Store',
       'Supermarket', 'Hotel Bar', 'Thrift / Vintage Store', 'Pet Store',
       'Flower Shop', 'Convenience Store', 'Gym', 'Train Station',
       'Shopping Mall', 'Metro Station', 'Bus Station', 'Park', 'Lake',
       'Pizza Place', 'Restaurant', 'Farm', 'Coffee Shop',
       'Scenic Lookout', 'Chinese Restaurant', 'Community Center',
       'Soccer Stadium', 'Indian Restaurant', 'Gas Station',
       'Scandinavian Restaurant', 'Movie Theater', 'Art Museum',
       'Sandwich Place', 'Sculpture Garden', 'Bakery', 'Asian Restaurant',
       'Italian Restaurant', 'Burger Joint', 'Mexican Restaurant',
       'Deli / Bodega', 'Playground'], dtype=object)

In [24]:
# check if the results contain "Bar"
"Bar" in venues_df['VenueCategory'].unique()

True

## Analyze Each Neighborhood

### Create a new DataFrame for Oslo Bars data only

In [26]:
# one hot encoding
oslo_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
oslo_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [oslo_onehot.columns[-1]] + list(oslo_onehot.columns[:-1])
oslo_onehot = oslo_onehot[fixed_columns]

print(oslo_onehot.shape)
oslo_onehot.head()


(840, 150)


,Neighborhoods,Advertising Agency,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Bakery,Bar,Basketball Court,Bathing Area,Beach,Beer Bar,Beer Garden,Beer Store,Bistro,Bookstore,Botanical Garden,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Café,Camera Store,Campground,Cemetery,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Community Center,Convenience Store,Creperie,Deli / Bodega,Department Store,Dumpling Restaurant,Electronics Store,Exhibit,Falafel Restaurant,Farm,Fast Food Restaurant,Fish Market,Flower Shop,Food,Food Court,Food Truck,French Restaurant,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,Gluten-free Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,History Museum,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Insurance Office,Italian Restaurant,Japanese Restaurant,Juice Bar,Lake,Light Rail Station,Lounge,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Movie Theater,Multiplex,Music Venue,Nightclub,Opera House,Palace,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pub,Racetrack,Rafting,Ramen Restaurant,Record Shop,Restaurant,Rock Club,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Ski Area,Ski Lodge,Ski Trail,Smoothie Shop,Soccer Field,Soccer Stadium,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Stables,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Yoga Studio
0,Alna,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,Alna,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Alna,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Alna,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Alna,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [27]:
oslo_grouped = oslo_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(oslo_grouped.shape)
oslo_grouped

(15, 150)


,Neighborhoods,Advertising Agency,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Bakery,Bar,Basketball Court,Bathing Area,Beach,Beer Bar,Beer Garden,Beer Store,Bistro,Bookstore,Botanical Garden,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Café,Camera Store,Campground,Cemetery,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Community Center,Convenience Store,Creperie,Deli / Bodega,Department Store,Dumpling Restaurant,Electronics Store,Exhibit,Falafel Restaurant,Farm,Fast Food Restaurant,Fish Market,Flower Shop,Food,Food Court,Food Truck,French Restaurant,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,Gluten-free Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,History Museum,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Insurance Office,Italian Restaurant,Japanese Restaurant,Juice Bar,Lake,Light Rail Station,Lounge,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Movie Theater,Multiplex,Music Venue,Nightclub,Opera House,Palace,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pub,Racetrack,Rafting,Ramen Restaurant,Record Shop,Restaurant,Rock Club,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Ski Area,Ski Lodge,Ski Trail,Smoothie Shop,Soccer Field,Soccer Stadium,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Stables,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Yoga Studio
0,Alna,0.000000,0.00,0.00,0.021277,0.000000,0.000000,0.000000,0.021277,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.021277,0.000000,0.00,0.000000,0.00,0.021277,0.000000,0.00,0.042553,0.00,0.000000,0.00,0.000,0.000000,0.00,0.00,0.000000,0.000000,0.000,0.021277,0.00,0.000000,0.000000,0.00,0.063830,0.00,0.00,0.000,0.000000,0.000000,0.021277,0.00,0.00,0.00,0.00,0.106383,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.106383,0.021277,0.042553,0.000000,0.000000,0.00,0.00,0.085106,0.021277,0.00,0.000,0.000000,0.000000,0.000000,0.00,0.021277,0.000000,0.00,0.00,0.000000,0.042553,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.021277,0.042553,0.00,0.021277,0.000000,0.000000,0.000000,0.000000,0.000000,0.021277,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.000,0.00,0.00,0.00,0.021277,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.021277,0.021277,0.00,0.000000,0.00,0.063830,0.021277,0.00,0.00,0.00,0.000000,0.00,0.021277,0.00,0.00,0.000000,0.000000,0.021277,0.00,0.00,0.00,0.00,0.00,0.021277,0.00
1,Bjerke,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.025000,0.000000,0.00,0.000000,0.00,0.025000,0.000000,0.00,0.050000,0.00,0.000000,0.00,0.025,0.000000,0.00,0.00,0.025000,0.000000,0.025,0.025000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.025,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.025000,0.000000,0.00,0.00,0.000000,0.00,0.225000,0.025000,0.025000,0.000000,0.000000,0.00,0.00,0.075000,0.025000,0.00,0.025,0.000000,0.000000,0.000000,0.00,0.025000,0.000000,0.00,0.00,0.000000,0.025000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.025000,0.000000,0.050000,0.000000,0.000000,0.000000,0.025000,0.000000,0.00,0.00,0.025000,0.00,0.00,0.00,0.00,0.025,0.00,0.00,0.00,0.075000,0.000000,0.000000,0.000000,0.00,0.000000,0.025000,0.00,0.0

In [28]:
len(oslo_grouped[oslo_grouped["Bar"] > 0])

6

In [30]:
num_top_venues = 5

for hood in oslo_grouped['Neighborhoods']:
    print("----"+hood+"----")
    temp = oslo_grouped[oslo_grouped['Neighborhoods'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alna----
                    venue  freq
0           Grocery Store  0.11
1  Furniture / Home Store  0.11
2                   Hotel  0.09
3       Electronics Store  0.06
4             Supermarket  0.06


----Bjerke----
           venue  freq
0  Grocery Store  0.22
1  Shopping Mall  0.08
2          Hotel  0.08
3    Pizza Place  0.05
4           Café  0.05


----Frogner----
                     venue  freq
0     Gym / Fitness Center  0.06
1  Scandinavian Restaurant  0.06
2                    Hotel  0.05
3           History Museum  0.04
4              Coffee Shop  0.04


----Gamle Oslo----
                venue  freq
0                 Bar  0.11
1         Coffee Shop  0.08
2                Park  0.04
3   Indian Restaurant  0.04
4  Italian Restaurant  0.03


----Grorud----
                venue  freq
0       Grocery Store  0.17
1                Lake  0.11
2  Athletics & Sports  0.11
3         Supermarket  0.06
4            Gym Pool  0.06


----Grünerløkka----
          venue  freq
0     

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
# Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhoods']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhoods'] = oslo_grouped['Neighborhoods']

for ind in np.arange(oslo_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(oslo_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alna,Furniture / Home Store,Grocery Store,Hotel,Supermarket,Electronics Store,Gym / Fitness Center,Park,Café,Metro Station,Pet Store
1,Bjerke,Grocery Store,Shopping Mall,Hotel,Pizza Place,Café,Gym / Fitness Center,Convenience Store,Soccer Stadium,Farm,Bus Station
2,Frogner,Scandinavian Restaurant,Gym / Fitness Center,Hotel,Bakery,Park,Burger Joint,Café,Wine Shop,History Museum,Coffee Shop
3,Gamle Oslo,Bar,Coffee Shop,Indian Restaurant,Park,Beer Bar,Italian Restaurant,Asian Restaurant,Café,Cocktail Bar,Sushi Restaurant
4,Grorud,Grocery Store,Athletics & Sports,Lake,Convenience Store,Fast Food Restaurant,Soccer Field,Asian Restaurant,Gym Pool,Gym / Fitness Center,Gym
5,Grünerløkka,Bar,Park,Coffee Shop,Cocktail Bar,Café,Brewery,Beer Bar,Italian Restaurant,Bakery,Restaurant
6,Nordre Aker,Bakery,Gym / Fitness Center,Sushi Restaurant,Grocery Store,Coffee Shop,Bar,Gym,Café,Deli / Bodega,Soccer Field
7,Nordstrand,Grocery Store,Metro Station,Beach,Supermarket,Furniture / Home Store,Gym / Fitness Center,Bus Station,Shopping Mall,Convenience Store,Movie Theater
8,Sagene,Park,Coffee Shop,Bakery,Bar,Cocktail Bar,Gym / Fitness Center,Café,Restaurant,Burger Joint,Indian Restaurant
9,St. Hanshaugen,Bar,Coffee Shop,Café,Park,Cocktail Bar,Scandinavian Restaurant,French Restaurant,Beer Bar,Restaurant,Brewery


### Create a new DataFrame for Oslo Bars data only

In [33]:
oslo_bars = oslo_grouped[["Neighborhoods","Bar"]]
oslo_bars.head()


,Neighborhoods,Bar
0,Alna,0.00
1,Bjerke,0.00
2,Frogner,0.01
3,Gamle Oslo,0.11
4,Grorud,0.00


In [34]:
# set number of clusters
kclusters = 5

oslo_clustering = oslo_bars.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(oslo_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 3, 0, 3, 2, 0, 4, 1], dtype=int32)

In [35]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
oslo_merged1 = oslo_bars.copy()

# add clustering labels
oslo_merged1["Cluster Labels"] = kmeans.labels_

In [36]:
oslo_merged1.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
oslo_merged1.head()

,Neighborhood,Bar,Cluster Labels
0,Alna,0.00,0
1,Bjerke,0.00,0
2,Frogner,0.01,0
3,Gamle Oslo,0.11,3
4,Grorud,0.00,0


In [37]:
oslo_mix = pd.merge(oslo_merged1, df, on='Neighborhood')

In [38]:
oslo_mix.head()

,Neighborhood,Bar,Cluster Labels,Latitude,Longitude
0,Alna,0.00,0,59.92680,10.84570
1,Bjerke,0.00,0,59.94221,10.81183
2,Frogner,0.01,0,59.91886,10.70239
3,Gamle Oslo,0.11,3,59.90624,10.77147
4,Grorud,0.00,0,59.96058,10.88492


In [40]:
oslo_mix.sort_values(["Cluster Labels"], inplace=True)
oslo_mix

,Neighborhood,Bar,Cluster Labels,Latitude,Longitude
0,Alna,0.000000,0,59.92680,10.84570
1,Bjerke,0.000000,0,59.94221,10.81183
2,Frogner,0.010000,0,59.91886,10.70239
4,Grorud,0.000000,0,59.96058,10.88492
7,Nordstrand,0.000000,0,59.86075,10.80260
10,Stovner,0.000000,0,59.96279,10.92083
11,Søndre Nordstrand,0.000000,0,59.83342,10.80778
12,Ullern,0.000000,0,59.92186,10.63724
13,Vestre Aker,0.000000,0,59.94805,10.64203
14,Østensjø,0.000000,0,59.87992,10.82584


### Finally, let's visualize the resulting clusters

In [41]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(oslo_mix['Latitude'], oslo_mix['Longitude'], oslo_mix['Neighborhood'], oslo_mix['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [43]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### Examine Clusters

In [44]:
oslo_mix.loc[oslo_mix['Cluster Labels'] == 0]

,Neighborhood,Bar,Cluster Labels,Latitude,Longitude
0,Alna,0.00,0,59.92680,10.84570
1,Bjerke,0.00,0,59.94221,10.81183
2,Frogner,0.01,0,59.91886,10.70239
4,Grorud,0.00,0,59.96058,10.88492
7,Nordstrand,0.00,0,59.86075,10.80260
10,Stovner,0.00,0,59.96279,10.92083
11,Søndre Nordstrand,0.00,0,59.83342,10.80778
12,Ullern,0.00,0,59.92186,10.63724
13,Vestre Aker,0.00,0,59.94805,10.64203
14,Østensjø,0.00,0,59.87992,10.82584


In [45]:
oslo_mix.loc[oslo_mix['Cluster Labels'] == 1]

,Neighborhood,Bar,Cluster Labels,Latitude,Longitude
9,St. Hanshaugen,0.09,1,59.92679,10.74111


In [46]:
oslo_mix.loc[oslo_mix['Cluster Labels'] == 2]

,Neighborhood,Bar,Cluster Labels,Latitude,Longitude
6,Nordre Aker,0.051724,2,59.94933,10.7314


In [47]:
oslo_mix.loc[oslo_mix['Cluster Labels'] == 3]

,Neighborhood,Bar,Cluster Labels,Latitude,Longitude
3,Gamle Oslo,0.11,3,59.90624,10.77147
5,Grünerløkka,0.11,3,59.92433,10.76119


In [48]:
oslo_mix.loc[oslo_mix['Cluster Labels'] == 4]

,Neighborhood,Bar,Cluster Labels,Latitude,Longitude
8,Sagene,0.07,4,59.93868,10.75763
